In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image data generator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values
    rotation_range=20, # Randomly rotate images
    width_shift_range=0.2, # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2, # Randomly shear images
    zoom_range=0.2, # Randomly zoom in on images
    horizontal_flip=True, # Randomly flip images horizontally
    validation_split=0.2 # Split data into train/validation sets
)

# Set image size and batch size
img_size = (224, 224) # Set image size to 224x224 pixels
batch_size = 32 # Set batch size to 32 images per batch

# Load images from folders and split into train/validation sets
train_data = train_datagen.flow_from_directory(
    'simple_images',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training' # Use subset parameter to get train set
)

val_data = train_datagen.flow_from_directory(
    'simple_images',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # Use subset parameter to get validation set
)

# Build model using transfer learning with pre-trained ResNet50V2 model
base_model = tf.keras.applications.ResNet50V2(
    include_top=False, # Remove top layer from pre-trained model
    input_shape=(224, 224, 3), # Set input shape to match image size
    weights='imagenet' # Use pre-trained ImageNet weights
)

# Freeze all layers in base model to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

# Add new top layer for classification
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(train_data.num_classes, activation='softmax')(x)

# Build final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model on training data and validate on validation data
history = model.fit(train_data, epochs=10, validation_data=val_data)

# Evaluate model on test data (optional)
# test_data = train_datagen.flow_from_directory(
#     'test_images',
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical'
# )

# test_loss, test_acc = model.evaluate(test_data)
# print('Test accuracy:', test_acc)


Found 4802 images belonging to 75 classes.
Found 1164 images belonging to 75 classes.
94668760/94668760 [==============================] - 3s 0us/step


/home/guillaume/miniconda3/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10


2023-03-27 17:54:02.454176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


151/151 [==============================] - ETA: 0s - loss: 3.3486 - accuracy: 0.2107

2023-03-27 17:55:22.082411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


151/151 [==============================] - 100s 649ms/step - loss: 3.3486 - accuracy: 0.2107 - val_loss: 2.6029 - val_accuracy: 0.3540
Epoch 2/10
151/151 [==============================] - 97s 647ms/step - loss: 2.3050 - accuracy: 0.4021 - val_loss: 2.3924 - val_accuracy: 0.3857
Epoch 3/10
151/151 [==============================] - 96s 634ms/step - loss: 1.8567 - accuracy: 0.5115 - val_loss: 2.1682 - val_accuracy: 0.4296
Epoch 4/10
151/151 [==============================] - 96s 635ms/step - loss: 1.6112 - accuracy: 0.5581 - val_loss: 2.1068 - val_accuracy: 0.4553
Epoch 5/10
151/151 [==============================] - 98s 647ms/step - loss: 1.3768 - accuracy: 0.6147 - val_loss: 2.1117 - val_accuracy: 0.4725
Epoch 6/10
151/151 [==============================] - 108s 719ms/step - loss: 1.2145 - accuracy: 0.6564 - val_loss: 2.1751 - val_accuracy: 0.4622
Epoch 7/10
151/151 [==============================] - 108s 715ms/step - loss: 1.0804 - accuracy: 0.6858 - val_loss: 2.1454 - val_accuracy: 

In [4]:
from tensorflow import keras
import numpy as np
img = tf.keras.preprocessing.image.load_img(
    'test_images/z1910141107270.jpg', target_size=(224, 224)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("predictions: ", predictions)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence.".format(dataset.class_names[np.argmax(score)], np.max(score))
)


1/1 [==============================] - 0s 20ms/step
predictions:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]
This image most likely belongs to {} with a {:.2f} percent confidence. 13 3.543199598789215
